In [61]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import rc
from scipy.signal import find_peaks 
from cProfile import label
from scipy.integrate import quad, trapezoid
import scipy.interpolate as scint

In [45]:
uvvis_path = "MnSpecies_uvvis.CSV"
uvvis=pd.read_csv(uvvis_path, sep = ';', decimal = ',')

peak_path = "MnSpecies_peak.CSV"
peak=pd.read_csv(peak_path, sep = ';', decimal = ',')

uvvis_water_path = "MnSpecies_water_uvvis.CSV"
uvvis_water=pd.read_csv(uvvis_water_path, sep = ';', decimal = ',')

peak_water_path = "MnSpecies_water_peak.CSV"
peak_water=pd.read_csv(peak_water_path, sep = ';', decimal = ',')

MnCl2aq_uvvis_path = "functional_uvvis.CSV"
MnCl2aq_uvvis=pd.read_csv(MnCl2aq_uvvis_path, sep = ';', decimal = ',')

MnCl2aq_peak_path = "functional_peak.CSV"
MnCl2aq_peak=pd.read_csv(MnCl2aq_peak_path, sep = ';', decimal = ',')

MnCl2aq_uvvis_water_path = "functional_uvvis_water.CSV"
MnCl2aq_uvvis_water=pd.read_csv(MnCl2aq_uvvis_water_path, sep = ';', decimal = ',')

MnCl2aq_peak_water_path = "functional_peak_water.CSV"
MnCl2aq_peak_water=pd.read_csv(MnCl2aq_peak_water_path, sep = ';', decimal = ',')

camb3lyp_uvvis_path = "camb3lyp_uvvis.CSV"
camb3lyp_uvvis=pd.read_csv(camb3lyp_uvvis_path, sep = ';', decimal = ',')

camb3lyp_peak_path = "camb3lyp_peak.CSV"
camb3lyp_peak=pd.read_csv(camb3lyp_peak_path, sep = ';', decimal = ',')

camb3lyp_uvvis_water_path = "camb3lyp_water_uvvis.CSV"
camb3lyp_uvvis_water=pd.read_csv(camb3lyp_uvvis_water_path, sep = ';', decimal = ',')

camb3lyp_peak_water_path = "camb3lyp_water_peak.CSV"
camb3lyp_peak_water=pd.read_csv(camb3lyp_peak_water_path, sep = ';', decimal = ',')

wb97xd_uvvis_path = "wb97xd_uvvis.CSV"
wb97xd_uvvis=pd.read_csv(wb97xd_uvvis_path, sep = ';', decimal = ',')

wb97xd_peak_path = "wb97xd_peak.CSV"
wb97xd_peak=pd.read_csv(wb97xd_peak_path, sep = ';', decimal = ',')

wb97xd_uvvis_water_path = "wb97xd_water_uvvis.CSV"
wb97xd_uvvis_water=pd.read_csv(wb97xd_uvvis_water_path, sep = ';', decimal = ',')

wb97xd_peak_water_path = "wb97xd_water_peak.CSV"
wb97xd_peak_water=pd.read_csv(wb97xd_peak_water_path, sep = ';', decimal = ',')

In [20]:
in_water_path = "cross_sec_MnCl2Short.csv"
in_water=pd.read_csv(in_water_path, index_col=0)

in_hcl_path = "cross_sec_inHClShort.csv"
in_hcl=pd.read_csv(in_hcl_path, index_col=0)

ForRep_path = "cross_sec_ForRepShort.csv"
ForRep=pd.read_csv(ForRep_path, index_col=0)
ForRep = ForRep.iloc[ForRep.index<730][:]

wNaCl_path = "cross_sec_wNaCl.csv"
wNaCl=pd.read_csv(wNaCl_path, index_col=0)

solar_path = 'actinic_flux_TUV.CSV' # calculated with TUV model: latitude=35   longitude=-41   Northern Atlantic Ocean
solar_irradiance = pd.read_csv(solar_path, sep = ';', decimal = ',')

no2_path = 'no2_rate_TUV.CSV'
no2_rate = pd.read_csv(no2_path, sep = ';', decimal = ',', index_col = 0)

In [4]:
def cross_section(epsilon):
    sigma = (1000*epsilon)/(6.022*10**23) #epsilon is in L/mol*cm so the multiplication by 1000 is conversion from L to cm**3
    return sigma

def makesigma(xvalues, yvalues):
    sigma = cross_section(yvalues.to_numpy())
    wavelength = xvalues.to_numpy()
    return pd.DataFrame(np.array([wavelength, sigma]).T, dtype = np.float64)

In [51]:
sigma_MnCl2wH2O = makesigma(MnCl2aq_uvvis["X b3lyp6311++"], MnCl2aq_uvvis['Y b3lyp6311++'])
sigma_MnCl2wH2O.columns = ['X MnCl2wH2O', 'Y MnCl2wH2O']
sigma_MnCl = makesigma(uvvis['X MnCl+'], uvvis['Y MnCl+'])
sigma_MnCl.columns = ['X MnCl+', 'Y MnCl+']
sigma_MnCl3 = makesigma(uvvis['X MnCl3-'], uvvis['Y MnCl3-'])
sigma_MnCl3.columns = ['X MnCl3-', 'Y MnCl3-']
sigma_MnClH2O = makesigma(uvvis['X MnClH2O'], uvvis['Y MnClH2O'])
sigma_MnClH2O.columns = ['X MnClH2O', 'Y MnClH2O']
sigma_MnCl2 = makesigma(uvvis['X MnCl2'], uvvis['Y MnCl2'])
sigma_MnCl2.columns = ['X MnCl2', 'Y MnCl2']
sigma_MnH2O = makesigma(uvvis['X MnH2O'], uvvis['Y MnH2O'])
sigma_MnH2O.columns = ['X MnH2O', 'Y MnH2O']
sigma_MnCl2wH2O_fit2 = makesigma(MnCl2aq_uvvis["X b3lyp fit2"], MnCl2aq_uvvis['Y b3lyp fit2'])
sigma_MnCl2wH2O_fit2.columns = ['X MnCl2wH2O fit2', 'Y MnCl2wH2O fit2']
sigma_MnCl_fit2 = makesigma(uvvis['X MnCl+ fit2'], uvvis['Y MnCl+ fit2'])
sigma_MnCl_fit2.columns = ['X MnCl+ fit2', 'Y MnCl+ fit2']
sigma_MnCl3_fit2 = makesigma(uvvis['X MnCl3- fit2'], uvvis['Y MnCl3- fit2'])
sigma_MnCl3_fit2.columns = ['X MnCl3- fit2', 'Y MnCl3- fit2']
sigma_MnClH2O_fit2 = makesigma(uvvis['X MnClH2O fit2'], uvvis['Y MnClH2O fit2'])
sigma_MnClH2O_fit2.columns = ['X MnClH2O fit2', 'Y MnClH2O fit2']
sigma_MnCl2_fit2 = makesigma(uvvis['X MnCl2 fit2'], uvvis['Y MnCl2 fit2'])
sigma_MnCl2_fit2.columns = ['X MnCl2 fit2', 'Y MnCl2 fit2']
sigma_MnH2O_fit2 = makesigma(uvvis['X MnH2O fit2'], uvvis['Y MnH2O fit2'])
sigma_MnH2O_fit2.columns = ['X MnH2O fit2', 'Y MnH2O fit2']

cross_sec_b3lyp = pd.concat([sigma_MnCl2wH2O, sigma_MnCl, sigma_MnCl3, sigma_MnClH2O, sigma_MnCl2, sigma_MnH2O, sigma_MnCl2wH2O_fit2,
                             sigma_MnCl_fit2, sigma_MnCl3_fit2, sigma_MnClH2O_fit2, sigma_MnCl2_fit2, sigma_MnH2O_fit2], axis=1)

cross_sec_b3lyp.to_csv('cross_sec_b3lyp.csv', index = False)

In [52]:
sigma_MnCl2wH2O = makesigma(MnCl2aq_uvvis_water["X b3lyp6311++"], MnCl2aq_uvvis_water['Y b3lyp6311++'])
sigma_MnCl2wH2O.columns = ['X MnCl2wH2O', 'Y MnCl2wH2O']
sigma_MnCl = makesigma(uvvis_water['X MnCl+'], uvvis_water['Y MnCl+'])
sigma_MnCl.columns = ['X MnCl+', 'Y MnCl+']
sigma_MnCl3 = makesigma(uvvis_water['X MnCl3-'], uvvis_water['Y MnCl3-'])
sigma_MnCl3.columns = ['X MnCl3-', 'Y MnCl3-']
sigma_MnClH2O = makesigma(uvvis_water['X MnClH2O'], uvvis_water['Y MnClH2O'])
sigma_MnClH2O.columns = ['X MnClH2O', 'Y MnClH2O']
sigma_MnCl2 = makesigma(uvvis_water['X MnCl2'], uvvis_water['Y MnCl2'])
sigma_MnCl2.columns = ['X MnCl2', 'Y MnCl2']
sigma_MnH2O = makesigma(uvvis_water['X MnH2O'], uvvis_water['Y MnH2O'])
sigma_MnH2O.columns = ['X MnH2O', 'Y MnH2O']
sigma_MnCl2wH2O_fit2 = makesigma(MnCl2aq_uvvis_water["X b3lyp fit2"], MnCl2aq_uvvis_water['Y b3lyp fit2'])
sigma_MnCl2wH2O_fit2.columns = ['X MnCl2wH2O fit2', 'Y MnCl2wH2O fit2']
sigma_MnCl_fit2 = makesigma(uvvis_water['X MnCl+ fit2'], uvvis_water['Y MnCl+ fit2'])
sigma_MnCl_fit2.columns = ['X MnCl+ fit2', 'Y MnCl+ fit2']
sigma_MnCl3_fit2 = makesigma(uvvis_water['X MnCl3- fit2'], uvvis_water['Y MnCl3- fit2'])
sigma_MnCl3_fit2.columns = ['X MnCl3- fit2', 'Y MnCl3- fit2']
sigma_MnClH2O_fit2 = makesigma(uvvis_water['X MnClH2O fit2'], uvvis_water['Y MnClH2O fit2'])
sigma_MnClH2O_fit2.columns = ['X MnClH2O fit2', 'Y MnClH2O fit2']
sigma_MnCl2_fit2 = makesigma(uvvis_water['X MnCl2 fit2'], uvvis_water['Y MnCl2 fit2'])
sigma_MnCl2_fit2.columns = ['X MnCl2 fit2', 'Y MnCl2 fit2']
sigma_MnH2O_fit2 = makesigma(uvvis_water['X MnH2O fit2'], uvvis_water['Y MnH2O fit2'])
sigma_MnH2O_fit2.columns = ['X MnH2O fit2', 'Y MnH2O fit2']

CrossSec_b3lyp_water = pd.concat([sigma_MnCl2wH2O, sigma_MnCl, sigma_MnCl3, sigma_MnClH2O, sigma_MnCl2, sigma_MnH2O, sigma_MnCl2wH2O_fit2,
                                  sigma_MnCl_fit2, sigma_MnCl3_fit2, sigma_MnClH2O_fit2, sigma_MnCl2_fit2, sigma_MnH2O_fit2], axis=1)

CrossSec_b3lyp_water.to_csv('CrossSec_b3lyp_water.csv', index = False)

In [53]:
sigma_MnCl2wH2O = makesigma(MnCl2aq_uvvis["X c-b6311++"], MnCl2aq_uvvis['Y c-b6311++'])
sigma_MnCl2wH2O.columns = ['X MnCl2wH2O', 'Y MnCl2wH2O']
sigma_MnCl = makesigma(camb3lyp_uvvis['X MnCl+'], camb3lyp_uvvis['Y MnCl+'])
sigma_MnCl.columns = ['X MnCl+', 'Y MnCl+']
sigma_MnCl3 = makesigma(camb3lyp_uvvis['X MnCl3-'], camb3lyp_uvvis['Y MnCl3-'])
sigma_MnCl3.columns = ['X MnCl3-', 'Y MnCl3-']
sigma_MnClH2O = makesigma(camb3lyp_uvvis['X MnClH2O'], camb3lyp_uvvis['Y MnClH2O'])
sigma_MnClH2O.columns = ['X MnClH2O', 'Y MnClH2O']
sigma_MnCl2 = makesigma(camb3lyp_uvvis['X MnCl2'], camb3lyp_uvvis['Y MnCl2'])
sigma_MnCl2.columns = ['X MnCl2', 'Y MnCl2']
sigma_MnH2O = makesigma(camb3lyp_uvvis['X MnH2O'], camb3lyp_uvvis['Y MnH2O'])
sigma_MnH2O.columns = ['X MnH2O', 'Y MnH2O']
sigma_MnCl2wH2O_fit2 = makesigma(MnCl2aq_uvvis["X c-b fit2"], MnCl2aq_uvvis['Y c-b fit2'])
sigma_MnCl2wH2O_fit2.columns = ['X MnCl2wH2O fit2', 'Y MnCl2wH2O fit2']
sigma_MnCl_fit2 = makesigma(camb3lyp_uvvis['X MnCl+ fit2'], camb3lyp_uvvis['Y MnCl+ fit2'])
sigma_MnCl_fit2.columns = ['X MnCl+ fit2', 'Y MnCl+ fit2']
sigma_MnCl3_fit2 = makesigma(camb3lyp_uvvis['X MnCl3- fit2'], camb3lyp_uvvis['Y MnCl3- fit2'])
sigma_MnCl3_fit2.columns = ['X MnCl3- fit2', 'Y MnCl3- fit2']
sigma_MnClH2O_fit2 = makesigma(camb3lyp_uvvis['X MnClH2O fit2'], camb3lyp_uvvis['Y MnClH2O fit2'])
sigma_MnClH2O_fit2.columns = ['X MnClH2O fit2', 'Y MnClH2O fit2']
sigma_MnCl2_fit2 = makesigma(camb3lyp_uvvis['X MnCl2 fit2'], camb3lyp_uvvis['Y MnCl2 fit2'])
sigma_MnCl2_fit2.columns = ['X MnCl2 fit2', 'Y MnCl2 fit2']
sigma_MnH2O_fit2 = makesigma(camb3lyp_uvvis['X MnH2O fit2'], camb3lyp_uvvis['Y MnH2O fit2'])
sigma_MnH2O_fit2.columns = ['X MnH2O fit2', 'Y MnH2O fit2']

cross_sec_camb3lyp = pd.concat([sigma_MnCl2wH2O, sigma_MnCl, sigma_MnCl3, sigma_MnClH2O, sigma_MnCl2, sigma_MnH2O, sigma_MnCl2wH2O_fit2,
                                sigma_MnCl_fit2, sigma_MnCl3_fit2, sigma_MnClH2O_fit2, sigma_MnCl2_fit2, sigma_MnH2O_fit2], axis=1)

cross_sec_camb3lyp.to_csv('cross_sec_camb3lyp.csv', index = False)

In [54]:
sigma_MnCl2wH2O = makesigma(MnCl2aq_uvvis_water["X c-b6311++"], MnCl2aq_uvvis_water['Y c-b6311++'])
sigma_MnCl2wH2O.columns = ['X MnCl2wH2O', 'Y MnCl2wH2O']
sigma_MnCl = makesigma(camb3lyp_uvvis_water['X MnCl+'], camb3lyp_uvvis_water['Y MnCl+'])
sigma_MnCl.columns = ['X MnCl+', 'Y MnCl+']
sigma_MnCl3 = makesigma(camb3lyp_uvvis_water['X MnCl3-'], camb3lyp_uvvis_water['Y MnCl3-'])
sigma_MnCl3.columns = ['X MnCl3-', 'Y MnCl3-']
sigma_MnClH2O = makesigma(camb3lyp_uvvis_water['X MnClH2O'], camb3lyp_uvvis_water['Y MnClH2O'])
sigma_MnClH2O.columns = ['X MnClH2O', 'Y MnClH2O']
sigma_MnCl2 = makesigma(camb3lyp_uvvis_water['X MnCl2'], camb3lyp_uvvis_water['Y MnCl2'])
sigma_MnCl2.columns = ['X MnCl2', 'Y MnCl2']
sigma_MnH2O = makesigma(camb3lyp_uvvis_water['X MnH2O'], camb3lyp_uvvis_water['Y MnH2O'])
sigma_MnH2O.columns = ['X MnH2O', 'Y MnH2O']
sigma_MnCl2wH2O_fit2 = makesigma(MnCl2aq_uvvis_water["X c-b fit2"], MnCl2aq_uvvis_water['Y c-b fit2'])
sigma_MnCl2wH2O_fit2.columns = ['X MnCl2wH2O fit2', 'Y MnCl2wH2O fit2']
sigma_MnCl_fit2 = makesigma(camb3lyp_uvvis_water['X MnCl+ fit2'], camb3lyp_uvvis_water['Y MnCl+ fit2'])
sigma_MnCl_fit2.columns = ['X MnCl+ fit2', 'Y MnCl+ fit2']
sigma_MnCl3_fit2 = makesigma(camb3lyp_uvvis_water['X MnCl3- fit2'], camb3lyp_uvvis_water['Y MnCl3- fit2'])
sigma_MnCl3_fit2.columns = ['X MnCl3- fit2', 'Y MnCl3- fit2']
sigma_MnClH2O_fit2 = makesigma(camb3lyp_uvvis_water['X MnClH2O fit2'], camb3lyp_uvvis_water['Y MnClH2O fit2'])
sigma_MnClH2O_fit2.columns = ['X MnClH2O fit2', 'Y MnClH2O fit2']
sigma_MnCl2_fit2 = makesigma(camb3lyp_uvvis_water['X MnCl2 fit2'], camb3lyp_uvvis_water['Y MnCl2 fit2'])
sigma_MnCl2_fit2.columns = ['X MnCl2 fit2', 'Y MnCl2 fit2']
sigma_MnH2O_fit2 = makesigma(camb3lyp_uvvis_water['X MnH2O fit2'], camb3lyp_uvvis_water['Y MnH2O fit2'])
sigma_MnH2O_fit2.columns = ['X MnH2O fit2', 'Y MnH2O fit2']

CrossSec_camb3lyp_water = pd.concat([sigma_MnCl2wH2O, sigma_MnCl, sigma_MnCl3, sigma_MnClH2O, sigma_MnCl2, sigma_MnH2O, sigma_MnCl2wH2O_fit2,
                                     sigma_MnCl_fit2, sigma_MnCl3_fit2, sigma_MnClH2O_fit2, sigma_MnCl2_fit2, sigma_MnH2O_fit2], axis=1)

CrossSec_camb3lyp_water.to_csv('CrossSec_camb3lyp_water.csv', index = False)

In [55]:
sigma_MnCl2wH2O = makesigma(MnCl2aq_uvvis["X wb6311++"], MnCl2aq_uvvis['Y wb6311++'])
sigma_MnCl2wH2O.columns = ['X MnCl2wH2O', 'Y MnCl2wH2O']
sigma_MnCl = makesigma(wb97xd_uvvis['X MnCl+'], wb97xd_uvvis['Y MnCl+'])
sigma_MnCl.columns = ['X MnCl+', 'Y MnCl+']
sigma_MnCl3 = makesigma(wb97xd_uvvis['X MnCl3-'], wb97xd_uvvis['Y MnCl3-'])
sigma_MnCl3.columns = ['X MnCl3-', 'Y MnCl3-']
sigma_MnClH2O = makesigma(wb97xd_uvvis['X MnClH2O'], wb97xd_uvvis['Y MnClH2O'])
sigma_MnClH2O.columns = ['X MnClH2O', 'Y MnClH2O']
sigma_MnCl2 = makesigma(wb97xd_uvvis['X MnCl2'], wb97xd_uvvis['Y MnCl2'])
sigma_MnCl2.columns = ['X MnCl2', 'Y MnCl2']
sigma_MnH2O = makesigma(wb97xd_uvvis['X MnH2O'], wb97xd_uvvis['Y MnH2O'])
sigma_MnH2O.columns = ['X MnH2O', 'Y MnH2O']
sigma_MnCl2wH2O_fit2 = makesigma(MnCl2aq_uvvis["X wb fit2"], MnCl2aq_uvvis['Y wb fit2'])
sigma_MnCl2wH2O_fit2.columns = ['X MnCl2wH2O fit2', 'Y MnCl2wH2O fit2']
sigma_MnCl_fit2 = makesigma(wb97xd_uvvis['X MnCl+ fit2'], wb97xd_uvvis['Y MnCl+ fit2'])
sigma_MnCl_fit2.columns = ['X MnCl+ fit2', 'Y MnCl+ fit2']
sigma_MnCl3_fit2 = makesigma(wb97xd_uvvis['X MnCl3- fit2'], wb97xd_uvvis['Y MnCl3- fit2'])
sigma_MnCl3_fit2.columns = ['X MnCl3- fit2', 'Y MnCl3- fit2']
sigma_MnClH2O_fit2 = makesigma(wb97xd_uvvis['X MnClH2O fit2'], wb97xd_uvvis['Y MnClH2O fit2'])
sigma_MnClH2O_fit2.columns = ['X MnClH2O fit2', 'Y MnClH2O fit2']
sigma_MnCl2_fit2 = makesigma(wb97xd_uvvis['X MnCl2 fit2'], wb97xd_uvvis['Y MnCl2 fit2'])
sigma_MnCl2_fit2.columns = ['X MnCl2 fit2', 'Y MnCl2 fit2']
sigma_MnH2O_fit2 = makesigma(wb97xd_uvvis['X MnH2O fit2'], wb97xd_uvvis['Y MnH2O fit2'])
sigma_MnH2O_fit2.columns = ['X MnH2O fit2', 'Y MnH2O fit2']

cross_sec_wb97xd = pd.concat([sigma_MnCl2wH2O, sigma_MnCl, sigma_MnCl3, sigma_MnClH2O, sigma_MnCl2, sigma_MnH2O, sigma_MnCl2wH2O_fit2,
                              sigma_MnCl_fit2, sigma_MnCl3_fit2, sigma_MnClH2O_fit2, sigma_MnCl2_fit2, sigma_MnH2O_fit2], axis=1)

cross_sec_wb97xd.to_csv('cross_sec_wb97xd.csv', index = False)

In [56]:
sigma_MnCl2wH2O = makesigma(MnCl2aq_uvvis_water["X wb6311++"], MnCl2aq_uvvis_water['Y wb6311++'])
sigma_MnCl2wH2O.columns = ['X MnCl2wH2O', 'Y MnCl2wH2O']
sigma_MnCl = makesigma(wb97xd_uvvis_water['X MnCl+'], wb97xd_uvvis_water['Y MnCl+'])
sigma_MnCl.columns = ['X MnCl+', 'Y MnCl+']
sigma_MnCl3 = makesigma(wb97xd_uvvis_water['X MnCl3-'], wb97xd_uvvis_water['Y MnCl3-'])
sigma_MnCl3.columns = ['X MnCl3-', 'Y MnCl3-']
sigma_MnClH2O = makesigma(wb97xd_uvvis_water['X MnClH2O'], wb97xd_uvvis_water['Y MnClH2O'])
sigma_MnClH2O.columns = ['X MnClH2O', 'Y MnClH2O']
sigma_MnCl2 = makesigma(wb97xd_uvvis_water['X MnCl2'], wb97xd_uvvis_water['Y MnCl2'])
sigma_MnCl2.columns = ['X MnCl2', 'Y MnCl2']
sigma_MnH2O = makesigma(wb97xd_uvvis_water['X MnH2O'], wb97xd_uvvis_water['Y MnH2O'])
sigma_MnH2O.columns = ['X MnH2O', 'Y MnH2O']
sigma_MnCl2wH2O_fit2 = makesigma(MnCl2aq_uvvis_water["X wb fit2"], MnCl2aq_uvvis_water['Y wb fit2'])
sigma_MnCl2wH2O_fit2.columns = ['X MnCl2wH2O fit2', 'Y MnCl2wH2O fit2']
sigma_MnCl_fit2 = makesigma(wb97xd_uvvis_water['X MnCl+ fit2'], wb97xd_uvvis_water['Y MnCl+ fit2'])
sigma_MnCl_fit2.columns = ['X MnCl+ fit2', 'Y MnCl+ fit2']
sigma_MnCl3_fit2 = makesigma(wb97xd_uvvis_water['X MnCl3- fit2'], wb97xd_uvvis_water['Y MnCl3- fit2'])
sigma_MnCl3_fit2.columns = ['X MnCl3- fit2', 'Y MnCl3- fit2']
sigma_MnClH2O_fit2 = makesigma(wb97xd_uvvis_water['X MnClH2O fit2'], wb97xd_uvvis_water['Y MnClH2O fit2'])
sigma_MnClH2O_fit2.columns = ['X MnClH2O fit2', 'Y MnClH2O fit2']
sigma_MnCl2_fit2 = makesigma(wb97xd_uvvis_water['X MnCl2 fit2'], wb97xd_uvvis_water['Y MnCl2 fit2'])
sigma_MnCl2_fit2.columns = ['X MnCl2 fit2', 'Y MnCl2 fit2']
sigma_MnH2O_fit2 = makesigma(wb97xd_uvvis_water['X MnH2O fit2'], wb97xd_uvvis_water['Y MnH2O fit2'])
sigma_MnH2O_fit2.columns = ['X MnH2O fit2', 'Y MnH2O fit2']

CrossSec_wb97xd_water = pd.concat([sigma_MnCl2wH2O, sigma_MnCl, sigma_MnCl3, sigma_MnClH2O, sigma_MnCl2, sigma_MnH2O, sigma_MnCl2wH2O_fit2,
                                   sigma_MnCl_fit2, sigma_MnCl3_fit2, sigma_MnClH2O_fit2, sigma_MnCl2_fit2, sigma_MnH2O_fit2], axis=1)

CrossSec_wb97xd_water.to_csv('CrossSec_wb97xd_water.csv', index = False)

In [ ]:
mean_wvl_list = (solar_irradiance['lower wvl'].values+solar_irradiance['upper wvl'].values)/2
solar_0km_spline = scint.interp1d(mean_wvl_list, solar_irradiance['0 km'].values, kind = 'linear')

In [76]:
def solar_exp(dataset, spline):
    solar = spline(dataset.index)
    new_dataset = dataset.mul(solar, axis = 0)
    return new_dataset

def exp_to_rate_per_lambda(exp_dataset, solar_dataset):
    mean_wvl = (solar_dataset['lower wvl'].values+solar_dataset['upper wvl'].values)/2
    photo_rate = {}
    for key in solar_dataset.keys()[2:]:
        spline = scint.interp1d(mean_wvl, solar_dataset[key].values, kind = 'linear')
        solar = solar_exp(exp_dataset, spline)
        solar.to_csv(f'ForRep_{key}.csv')
        photo_rate[key] = []
        for s_key in solar.keys():
            pr = -trapezoid(solar[s_key].values, solar.index) # minus for reverse integration direction
            photo_rate[key] += [pr]
    photo_df = pd.DataFrame(photo_rate, index = solar.keys()).T
    return photo_df

def solar_0km_cal(dataset):
    new_dataset = dataset.copy()
    for x_key, y_key in zip(dataset.keys()[::2], dataset.keys()[1::2]):
        mask = dataset[x_key]>121
        solar = solar_0km_spline(dataset[x_key][mask])
        solar_cross = solar*dataset[y_key][mask]
        full_solar_cross = np.append([0 for i in range(np.sum(~mask))], solar_cross)
        new_dataset[y_key] = full_solar_cross
    return new_dataset

def solar_cal_plot(dataset, labels):
    fig, ax = plt.subplots()
    for x_key, y_key, label in zip(dataset.keys()[::2], dataset.keys()[1::2], labels):
        sns.lineplot(x = x_key, y = y_key, data = dataset, ax = ax, label = label)
    plt.show()



In [ ]:
rate_matrix = exp_to_rate_per_lambda(ForRep, solar_irradiance)
rate_matrix.to_csv('rate_matrix.csv')

display(rate_matrix)

In [58]:
labels_cal = [r'[MnCl$_{2}$(H$_{2}$O)$_{4}$]', r'[MnCl]$^{+}$', r'[MnCl$_{3}$]$^{-}$', r'[MnCl(H$_{2}$O)$_{5}$]$^{+}$', r'MnCl$_{2}$', r'[Mn(H$_{2}$O)$_{6}$]$^{2+}$']

b3lyp_0km = solar_0km_cal(cross_sec_b3lyp)

In [5]:
NaCl_wMn_path = 'NaCl_wMn_25deg.csv'
NaCl_wMn = pd.read_csv(NaCl_wMn_path)

SW_wMn_path = 'seawater_wMn_25deg.csv'
SW_wMn = pd.read_csv(SW_wMn_path)

In [ ]:
# Function that sums all spectra while taking species percent into account

def total_spec_simpel(wavelength, spec, per_Mn):
    wavelength_extension_high = np.linspace(max(wavelength)+0.1, 700)
    wavelength_extension_low = np.linspace(0, min(wavelength)-0.1)
    wavelength_full = pd.concat([wavelength_extension_low, wavelength, wavelength_extension_high])
    spec_extension_high = pd.DataFrame(np.zeros_like(wavelength_extension_high))
    spec_extension_low = pd.DataFrame(np.zeros_like(wavelength_extension_low))
    Mn_spec = pd.concat([spec_extension_low, spec, spec_extension_high], index = wavelength_full)
    Mn_scaled = Mn_spec*per_Mn
    return pd.DataFrame(Mn_scaled)